<a href="https://colab.research.google.com/github/NamasteAI/ToxicCommentsClassifier/blob/main/ToxicCommentClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import required libraries

In [8]:
import pandas as pd
import tensorflow as tf
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
df = pd.read_csv('/content/drive/MyDrive/KaggleToxicCommentDataset/train.csv')

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# Preprocessing

In [11]:
from tensorflow.keras.layers import TextVectorization

In [18]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [19]:
MAX_FEATURES = 200000

In [20]:
vectorizer = TextVectorization(max_tokens = MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [21]:
vectorizer.adapt(X.values)

In [27]:
vectorizer('hello, how are you?')

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([288,  73,  20, ...,   0,   0,   0])>

In [28]:
vectorizer_text= vectorizer(X.values)

In [29]:
vectorizer_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [31]:
dataset = tf.data.Dataset.from_tensor_slices((vectorizer_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [33]:
batchX, batchy = dataset.as_numpy_iterator().next()

In [35]:
train = dataset.take(int(len(dataset)* .7))
val = dataset.skip(int(len(dataset)* .7)).take(int(len(dataset)* .2))
test = dataset.skip(int(len(dataset)* .9)).take(int(len(dataset)* .1))

In [37]:
train_generator=train.as_numpy_iterator()

In [41]:
train_generator.next()

(array([[ 165,  148,    8, ...,    0,    0,    0],
        [1376,  315,    8, ...,    0,    0,    0],
        [  70, 1926,   18, ...,    0,    0,    0],
        ...,
        [   7,   52,  132, ...,    0,    0,    0],
        [  26,    3,  855, ...,    0,    0,    0],
        [ 326,  267,  171, ...,    0,    0,    0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

# Create Sequential Model

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [47]:
model = Sequential()
mMAX_FEATURESodel.add(Embedding(+1, 32))
model.add(Bidirectional(LSTM(32, activation= 'tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation= 'relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [48]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [49]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                16640     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [ ]:
history = model.fit(train, epochs=1, validation_data=val)

1617/6981 [=====>........................] - ETA: 2:22:26 - loss: 0.0855